In [27]:

with open("../data/the_verdict.txt", "r") as f:
    raw_text = f.read()

print(f"Total characters: {len(raw_text)}")
print(f"Total words: {len(raw_text.split())}")

Total characters: 20479
Total words: 3634


In [28]:
import re

text = "Hello world this is some sample text."
result = re.split(r"(\s)", text)
print(result)

['Hello', ' ', 'world', ' ', 'this', ' ', 'is', ' ', 'some', ' ', 'sample', ' ', 'text.']


In [29]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ' ', 'world', ' ', 'this', ' ', 'is', ' ', 'some', ' ', 'sample', ' ', 'text', '.', '']


we need to have spaces, commas and fullstops as tokens as well.

In [30]:
result = [item for item in result if item.strip()]
print(result)

['Hello', 'world', 'this', 'is', 'some', 'sample', 'text', '.']


In [31]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [32]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [33]:
print(len(preprocessed))

4690


## Converting tokens into token ID

In [34]:
all_words = sorted(set(preprocessed))

vocab = {word: i for i, word in enumerate(all_words)}

In [35]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [36]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [37]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

Special tokens:

<|unk|> - unknown token
<|endoftext|> - end of text

In [38]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [39]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int.get(s, self.str_to_int["<|unk|>"]) for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str.get(i, "<|unk|>") for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [40]:
text_str_1 = "hello world this is a test"
text_str_2 = "hello world this is a test. This is a test, too."
text_str = "|<endoftext>|".join((text_str_1, text_str_2))
print(text_str)

tokenizer = SimpleTokenizerV2(vocab)
ids = tokenizer.encode(text_str)
print(ids)

hello world this is a test|<endoftext>|hello world this is a test. This is a test, too.
[1131, 1131, 999, 584, 115, 1131, 1131, 999, 584, 115, 1131, 7, 97, 584, 115, 1131, 5, 1020, 7]


In [41]:
tokenizer.decode(ids)

'<|unk|> <|unk|> this is a <|unk|> <|unk|> this is a <|unk|>. This is a <|unk|>, too.'